<a href="https://colab.research.google.com/github/sindhusatish707/Exploring_Data_Engineering/blob/day_1/Exploring_Data_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Day 1

In [1]:
import requests
import pandas as pd

In [3]:
url = "https://jsonplaceholder.typicode.com/posts"
response = requests.get(url)
data = response.json()

In [8]:
df = pd.DataFrame(data)
df = df[['userId', 'id', 'title']]
df = df.rename(columns={'id': 'postId'})

In [9]:
df

,userId,postId,title
0,1,1,sunt aut facere repellat provident occaecati e...
1,1,2,qui est esse
2,1,3,ea molestias quasi exercitationem repellat qui...
3,1,4,eum et est occaecati
4,1,5,nesciunt quas odio
...,...,...,...
95,10,96,quaerat velit veniam amet cupiditate aut numqu...
96,10,97,quas fugiat ut perspiciatis vero provident
97,10,98,laboriosam dolor voluptates
98,10,99,temporibus sit alias delectus eligendi possimu...


index=False → tells pandas not to write the row indices (the numbers 0, 1, 2, …) into the CSV file

In [10]:
df.to_csv('posts_clean.csv', index=False)

In [11]:
df.head()

,userId,postId,title
0,1,1,sunt aut facere repellat provident occaecati e...
1,1,2,qui est esse
2,1,3,ea molestias quasi exercitationem repellat qui...
3,1,4,eum et est occaecati
4,1,5,nesciunt quas odio
